In [3]:
# importem biblioteques
!pip install jovian --upgrade --quiet
import requests
from bs4 import BeautifulSoup
import jovian
import pandas as pd
import numpy as np
import scrapy
import logging
import json
import sys
import re
from scrapy.http import TextResponse
from scrapy.crawler import CrawlerProcess
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Tasca M10 T01

**EXERCICI 1**

**Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i selenium després**

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

## Beautiful Soup amb quotes.toscrape.com

In [4]:
# descarreguem la pàgina
url = "http://quotes.toscrape.com"
page = requests.get(url)

print(page.text)

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    

<div class="row">
    <div class="col-md-8">

    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
        <span>by <small class="author" itempr

In [5]:
# mirem si la pàgina s'ha descarregat correctament, si el resultat es troba
# entre 200 y 209 voldrà dir que s'ha descarregat correctament.
page.status_code

200

In [6]:
# per extraure informació del codi font HTML mitjançant programació
doc = BeautifulSoup(page.text, 'html.parser')
type(doc)

bs4.BeautifulSoup

In [7]:
# si mirem el cuadro de desenvolupadors a chrome veiem una etiqueta div amb la
# classe establerta en quote. busquem totes les etiquetes div que tinguin
# la clase quote.b
div_tags = doc.find_all('div', class_ = 'quote')
len(div_tags)

10

In [8]:
# cada pàgina té 10 cites, per tant el length de div_tags és 10. 
# div_tags conté informació de la cita, el nom de l'autor, enllaços a la 
# bibliografía de l'autor
def get_quotes(div_tags):
# d'aquesta manera obtenim la llista de cites per una pàgina
    quotes = []
    for tag in div_tags:
        quote = tag.find('span', class_ = 'text').text
        quotes.append(quote)
    return quotes

In [9]:
get_quotes(div_tags)

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

In [10]:
# ara extraiem els noms dels autors que es troba dins de small tag
def get_author_name(div_tags):
    authors = []
    for tag in div_tags:
        span_tag = tag.find('span', class_ = None)
        author = span_tag.find('small', class_ = 'author').text
        authors.append(author)
    return authors

In [11]:
get_author_name(div_tags)

['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']

In [12]:
# extreiem
def get_quote_tag(div_tags):
    name_tags = []
    for tag in div_tags:
        name_tag = tag.find('div', class_ = 'tags').meta['content']
        name_tags.append(name_tag)
    return name_tags

In [13]:
get_quote_tag(div_tags)

['change,deep-thoughts,thinking,world',
 'abilities,choices',
 'inspirational,life,live,miracle,miracles',
 'aliteracy,books,classic,humor',
 'be-yourself,inspirational',
 'adulthood,success,value',
 'life,love',
 'edison,failure,inspirational,paraphrased',
 'misattributed-eleanor-roosevelt',
 'humor,obvious,simile']

In [14]:
# extreiem els enllaços de la biografia de l'autor.
def get_author_urls(div_tags):
    author_links = []
    for tag in div_tags:
        span_tag = tag.find('span', class_ = None)
        author_link = 'http://quotes.toscrape.com'+span_tag.find('a')['href']
        author_links.append(author_link)
    return author_links

In [15]:
get_author_urls(div_tags)

['http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/J-K-Rowling',
 'http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/Jane-Austen',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/Andre-Gide',
 'http://quotes.toscrape.com/author/Thomas-A-Edison',
 'http://quotes.toscrape.com/author/Eleanor-Roosevelt',
 'http://quotes.toscrape.com/author/Steve-Martin']

In [16]:
author_names = get_author_name(div_tags)
quotes_list =  get_quotes(div_tags)
urls = get_author_urls(div_tags)
subject_names = get_quote_tag(div_tags)

In [17]:
# fem un diccionari combinant tota la llista obtinguda al analitzar el lloc web.
def list_of_dict(quotes_list, author_names, urls, subject_names):
    return[{'Quotes' : quotes_list[i],
            'Author' : author_names[i],
            'Tags' : subject_names[i],
            'Link' : urls[i]} for i in range(len(quotes_list))]

In [18]:
quotes_dict = list_of_dict(quotes_list,author_names,urls,subject_names)

In [19]:
# convertim l'arxiu en csv
df = pd.DataFrame(quotes_dict)
df.to_csv('cites.csv', index = None)

In [20]:
# mirem l'arxiu
pd.read_csv('cites.csv')

,Quotes,Author,Tags,Link
0,“The world as we have created it is a process ...,Albert Einstein,"change,deep-thoughts,thinking,world",http://quotes.toscrape.com/author/Albert-Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities,choices",http://quotes.toscrape.com/author/J-K-Rowling
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational,life,live,miracle,miracles",http://quotes.toscrape.com/author/Albert-Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy,books,classic,humor",http://quotes.toscrape.com/author/Jane-Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself,inspirational",http://quotes.toscrape.com/author/Marilyn-Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein,"adulthood,success,value",http://quotes.toscrape.com/author/Albert-Einstein
6,“It is better to be hated for what you are tha...,André Gide,"life,love",http://quotes.toscrape.com/author/Andre-Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"edison,failure,inspirational,paraphrased",http://quotes.toscrape.com/author/Thomas-A-Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,misattributed-eleanor-roosevelt,http://quotes.toscrape.com/author/Eleanor-Roos...
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"humor,obvious,simile",http://quotes.toscrape.com/author/Steve-Martin


## Selenium amb quotes.to.scrape.com

In [21]:
# opcions de navegació
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--distable-extensions')
driver_path = 'C:\\chromedriver.exe'
driver = webdriver.Chrome(driver_path, chrome_options = options)
# així s'obrirà el navegador
driver.get('http://quotes.toscrape.com')

total = []

cites = driver.find_elements(By.CLASS_NAME,"quote")

for quote in cites:
    cita = quote.find_element(By.CLASS_NAME,'text').text[1:-2]
    autor = quote.find_element(By.CLASS_NAME,'author').text
    tags = quote.find_element(By.CLASS_NAME,'tags').text[6:]
    new = ((cita, autor, tags))
    total.append(new)

'''for x in range (len(cites)):
    print(cites[x].text)'''

driver.quit()

df = pd.DataFrame(total, columns=['cita','autor','tags'])
df.to_csv('cites_selenium.csv', index = None)
df.head(10)

C:\Users\imber\AppData\Local\Temp\ipykernel_15772\1881963963.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options = options)
C:\Users\imber\AppData\Local\Temp\ipykernel_15772\1881963963.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options = options)


,cita,autor,tags
0,The world as we have created it is a process o...,Albert Einstein,change deep-thoughts thinking world
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,abilities choices
2,There are only two ways to live your life. One...,Albert Einstein,inspirational life live miracle miracles
3,"The person, be it gentleman or lady, who has n...",Jane Austen,aliteracy books classic humor
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,be-yourself inspirational
5,Try not to become a man of success. Rather bec...,Albert Einstein,adulthood success value
6,It is better to be hated for what you are than...,André Gide,life love
7,"I have not failed. I've just found 10,000 ways...",Thomas A. Edison,edison failure inspirational paraphrased
8,A woman is like a tea bag; you never know how ...,Eleanor Roosevelt,misattributed-eleanor-roosevelt
9,"A day without sunshine is like, you know, night",Steve Martin,humor obvious simile


**Amb l'exercici de wikipedia vaig a obtenir el resultat final de la classificació del mundial d'aquest any de Formula 1**

## Beautiful Soup amb wikipedia

In [22]:
# descarreguem la pàgina
url = "https://es.wikipedia.org/wiki/Temporada_2022_de_F%C3%B3rmula_1"
page = requests.get(url)
page.status_code
page_text = page.text
# demanem que ens digui si l'extracció del text s'ha fet correctament, si recordem
# de 200 a 208 s'ha fet correctament
print(page.status_code)

200


In [23]:
soup = BeautifulSoup(page_text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Temporada 2022 de Fórmula 1 - Wikipedia, la enciclopedia libre
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo","junio","julio","agosto","septiembre","octubre","noviembre","diciembre"],"wgRequestId":"2d494288-d14b-4be6-8036-475d1cff158e","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Temporada_2022_de_Fórmula_1","wgTitle":"Temporada 2022 de Fórmula 1","wgCurRevisionId":147806806,"wgRevisionId":147806806,"wgArticleId":9854735,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Páginas con plantillas con argumentos duplicados","

In [24]:
#demanem que ens retorni totes les taules
soup.find_all('table')

[<table class="infobox" style="width:22.7em; line-height: 1.4em; text-align:left; padding:.23em;"><tbody><tr><th class="cabecera automovilismo" colspan="3" style="text-align:center;background-color:#99ccff;color:black;">Temporada 2022 de Fórmula 1</th></tr><tr><td colspan="3" style="text-align:center;background-color:#99ccff;;">
 <i><b><a class="mw-redirect" href="/wiki/Campeonato_Mundial_de_F%C3%B3rmula_1" title="Campeonato Mundial de Fórmula 1">Campeonato Mundial de Fórmula 1</a> de la <a href="/wiki/Federaci%C3%B3n_Internacional_del_Autom%C3%B3vil" title="Federación Internacional del Automóvil">FIA</a></b></i></td></tr><tr><td class="imagen" colspan="3" style="text-align:center;font-size: 92%;;">
 <a class="image" href="/wiki/Archivo:Max_Verstappen_2017_Malaysia_3.jpg"><img alt="Max Verstappen 2017 Malaysia 3.jpg" data-file-height="1200" data-file-width="800" decoding="async" height="368" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/75/Max_Verstappen_2017_Malaysia_3.jpg/245

In [25]:
# demanem quantes taules hi ha a l'article
len(soup.find_all('table'))

23

In [26]:
# segons les característiques hi ha dues possibles taules.
print('Clases de cada taula:')
for table in soup.find_all('table'):
    print(table.get('class'))

Clases de cada taula:
['infobox']
['col1izq', 'col2izq']
None
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
['wikitable']
None
['wikitable']
['wikitable']
None
['wikitable']
['wikitable']
['wikitable', 'sortable']
['wikitable', 'sortable']
['hlist', 'navbox-inner']


In [27]:
# busco la taula de classificació
right_table = soup.find_all('table', class_= 'wikitable sortable')
# mirem les taules i selecciono la taula [0] que és la que vull
right_table = right_table[0]
right_table

<table class="wikitable sortable" style="text-align:center; font-size: 80%">
<tbody><tr>
<th>Pos.
</th>
<th>Piloto
</th>
<th>Escudería
</th>
<th>Grandes Premios
</th>
<th>Victorias
</th>
<th>Podios
</th>
<th><i>Poles</i>
</th>
<th>Vueltas rápidas
</th>
<th>Vueltas lideradas
</th>
<th>Puntos
</th></tr>
<tr>
<th>1
</th>
<td align="left"><span class="flagicon"><a class="image" href="/wiki/Archivo:Flag_of_the_Netherlands.svg" title="Bandera de los Países Bajos"><img alt="Bandera de los Países Bajos" class="thumbborder" data-file-height="600" data-file-width="900" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_the_Netherlands.svg/20px-Flag_of_the_Netherlands.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_the_Netherlands.svg/30px-Flag_of_the_Netherlands.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_the_Netherlands.svg/40px-Flag_of_the_Netherlands.svg.png 2x" width="20"/></a></span> <a hre

In [28]:
# llegim l'html
df = pd.read_html(str(right_table))
df= pd.DataFrame(df[0])
# elimino les columnes que no vull
df_1 = df.drop(['Victorias','Podios','Poles','Vueltas rápidas', 'Vueltas lideradas'], axis = 1)
# converteixo en csv
df_1.to_csv('classificacio.csv', index = None)
# llegeixo el cs
pd.read_csv('classificacio.csv')

,Pos.,Piloto,Escudería,Grandes Premios,Puntos
0,1,Max Verstappen,Red Bull,22,454
1,2,Charles Leclerc,Ferrari,22,308
2,3,Sergio Pérez,Red Bull,22,305
3,4,George Russell,Mercedes,22,275
4,5,Carlos Sainz Jr.,Ferrari,22,246
5,6,Lewis Hamilton,Mercedes,22,240
6,7,Lando Norris,McLaren,22,122
7,8,Esteban Ocon,Alpine,22,92
8,9,Fernando Alonso,Alpine,22,81
9,10,Valtteri Bottas,Alfa Romeo,22,49


## Selenium amb wikipedia

In [29]:
# opcions de navegació
dfbase = pd.DataFrame()
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--distable-extensions')
driver_path = 'C:\\chromedriver.exe'
driver = webdriver.Chrome(driver_path, chrome_options = options)
# així s'obrirà el navegador
driver.get('https://es.wikipedia.org/wiki/Temporada_2022_de_F%C3%B3rmula_1')
# extrec la taula
item = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[14]')
# extrec les dades de la taula
for table in item:
    data = [item.text for item in table.find_elements(By.XPATH, ".//*[self::td or self::th]")]
    print(data)
# així es tancarà el navegador
driver.quit()

C:\Users\imber\AppData\Local\Temp\ipykernel_15772\2776092462.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options = options)
C:\Users\imber\AppData\Local\Temp\ipykernel_15772\2776092462.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options = options)


['Pos.', 'Piloto', 'Escudería', 'Grandes Premios', 'Victorias', 'Podios', 'Poles', 'Vueltas rápidas', 'Vueltas lideradas', 'Puntos', '1', 'Max Verstappen', 'Red Bull', '22', '15', '17', '7', '5', '498', '454', '2', 'Charles Leclerc', 'Ferrari', '22', '3', '11', '9', '3', '310', '308', '3', 'Sergio Pérez', 'Red Bull', '22', '2', '11', '1', '3', '147', '305', '4', 'George Russell', 'Mercedes', '22', '1', '8', '1', '4', '28', '275', '5', 'Carlos Sainz Jr.', 'Ferrari', '22', '1', '9', '3', '2', '72', '246', '6', 'Lewis Hamilton', 'Mercedes', '22', '', '9', '', '2', '36', '240', '7', 'Lando Norris', 'McLaren', '22', '', '1', '', '2', '', '122', '8', 'Esteban Ocon', 'Alpine', '22', '', '', '', '', '', '92', '9', 'Fernando Alonso', 'Alpine', '22', '', '', '', '', '1', '81', '10', 'Valtteri Bottas', 'Alfa Romeo', '22', '', '', '', '', '', '49', '11', 'Daniel Ricciardo', 'McLaren', '22', '', '', '', '', '', '37', '12', 'Sebastian Vettel', 'Aston Martin Aramco', '20', '', '', '', '', '2', '37', 

In [30]:
columns = ['Pos',
           'Piloto',
           'Escuderia',
           'Grandes Premios',
           'Victorias',
           'Podios',
           'Poles',
           'Vueltas Rapidas',
           'Vueltas lideradas',
           'Puntos']
columnes = len(columns)
print(columnes)

10


In [31]:
nnn = []
for i in range(0, len(data), columnes):
    nnn.append(data[i:i + columnes])
    
df_selenium = pd.DataFrame(nnn, columns = columns)
# converteixo en csv
df_selenium.to_csv('classificacio_selenium.csv', index = None)
# llegeixo el cs
pd.read_csv('classificacio_selenium.csv')
# elimino les columnes que no vull
df_1_selenium = df_selenium.drop(columns = ['Victorias','Podios','Poles','Vueltas Rapidas','Vueltas lideradas'])
# elimino la primera fila que son les columnes duplicades
df_2_selenium = df_1_selenium.drop(0) 
# imprimeixo
df_2_selenium

,Pos,Piloto,Escuderia,Grandes Premios,Puntos
1,1,Max Verstappen,Red Bull,22,454
2,2,Charles Leclerc,Ferrari,22,308
3,3,Sergio Pérez,Red Bull,22,305
4,4,George Russell,Mercedes,22,275
5,5,Carlos Sainz Jr.,Ferrari,22,246
6,6,Lewis Hamilton,Mercedes,22,240
7,7,Lando Norris,McLaren,22,122
8,8,Esteban Ocon,Alpine,22,92
9,9,Fernando Alonso,Alpine,22,81
10,10,Valtteri Bottas,Alfa Romeo,22,49


**EXERCICI 3**

**Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després.**

## Selenium

In [32]:
# opcions de navegació
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--distable-extensions')
driver_path = 'C:\\chromedriver.exe'
driver = webdriver.Chrome(driver_path, chrome_options = options)
# així s'obrirà el navegador
driver.get('https://datosmacro.expansion.com/materias-primas/petroleo')# opcions de navegació
# accepto cookies
element = driver.find_element(By.XPATH,'//*[@id="didomi-notice-agree-button"]')
element.click()
# extrec la informació:
nom = driver.find_element(By.XPATH,'//*[@id="tb1_43803"]/tbody/tr[1]/td[1]').text[:-3]
cotització = driver.find_element(By.XPATH,'//*[@id="tb1_43803"]/tbody/tr[1]/td[3]').text
data = driver.find_element(By.XPATH,'//*[@id="tb1_43803"]/tbody/tr[1]/td[2]').text
print('el',nom,'cotitza a',cotització,'amb data',data)
# així es tancarà el navegador
driver.quit()

C:\Users\imber\AppData\Local\Temp\ipykernel_15772\2263319879.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, chrome_options = options)
C:\Users\imber\AppData\Local\Temp\ipykernel_15772\2263319879.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(driver_path, chrome_options = options)


el Precio petróleo Brent europeo  cotitza a 80,14 $ amb data 13/12/2022


## Scrapy

In [41]:
class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('resultados.jl','w')
    def close_spider(self,spider):
        self.file.close()
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [42]:
# defineixo l'spider
class BrentSpider(scrapy.Spider):
# nom de l'spider
    name = "Brent"
# url que buscarà
    start_urls = [
        'https://datosmacro.expansion.com/materias-primas/petroleo',
    ]
    custom_settings = { # com es guardarà l'arxiu
        'LOG_LEVEL' : logging.WARNING,
        'FEED_FORMAT' : 'csv',
        'FEED_URI' : 'brent.csv'
    }
# defineixo lo que vull extreure, en format text dins de la taula
    def parse(self,response):
        for row in response.xpath('//*[@id="tb1_43803"]'):
            yield{
                'nom' : row.xpath('//*[@id="tb1_43803"]/tbody/tr[1]/td[1]/a//text()').extract_first(),
                'cotització' : row.xpath('//*[@id="tb1_43803"]/tbody/tr[1]/td[3]//text()').extract_first(),
                'data' : row.xpath('//*[@id="tb1_43803"]/tbody/tr[1]/td[2]//text()').extract_first(),
            }
    

In [43]:
# si no introdueixo aquest condicional surt un missatge d'error on diu que ja s'ha creat el procediment
if "twisted.internet.reactor" in sys.modules:
    del sys.modules["twisted.internet.reactor"]
# executo l'spider    
process = CrawlerProcess()
process.crawl(BrentSpider)
process.start()

2022-12-14 18:34:59 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-12-14 18:34:59 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 3.4.8, Platform Windows-10-10.0.22621-SP0
2022-12-14 18:34:59 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}


In [45]:
# un cop s'ha executat l'spider tindré un arxiu .csv al ordinador, si el converteixo en DataFrame, haig d'eliminar la segona
# fila que són els headers i la primera que inclou llenguatge html.
# llegeido l'arxiu.
df = pd.read_csv('C:\Brent.csv')
df1 = df.drop([0,1], axis = 0)
df1.head()

,nom,cotització,data
2,Precio petróleo Brent europeo [+],"80,14 $",13/12/2022
